In [ ]:
pip install riotwatcher

In [2]:
import pandas as pd
import requests
import datetime
import base64
import os
import pandas as pd
import re
import datetime as dt
from riotwatcher import LolWatcher, ApiError
from pandas import json_normalize

In [ ]:
lol_watcher = LolWatcher('RGAPI-beefeee6-bcef-480e-bcdc-12d3a372ece0')

my_region = 'na1'

me = lol_watcher.summoner.by_name(my_region, 'Doublelift')
print(me)

# all objects are returned (by default) as a dict
# lets see if i got diamond yet (i probably didnt)
my_ranked_stats = lol_watcher.league.by_summoner(my_region, me['id'])
print(my_ranked_stats)

# First we get the latest version of the game from data dragon
versions = lol_watcher.data_dragon.versions_for_region(my_region)
champions_version = versions['n']['champion']

# Lets get some champions
current_champ_list = lol_watcher.data_dragon.champions(champions_version)
print(current_champ_list)

# For Riot's API, the 404 status code indicates that the requested data wasn't found and
# should be expected to occur in normal operation, as in the case of a an
# invalid summoner name, match ID, etc.
#
# The 429 status code indicates that the user has sent too many requests
# in a given amount of time ("rate limiting").

try:
    response = lol_watcher.summoner.by_name(my_region, 'this_is_probably_not_anyones_summoner_name')
except ApiError as err:
    if err.response.status_code == 429:
        print('We should retry in {} seconds.'.format(err.headers['Retry-After']))
        print('this retry-after is handled by default by the RiotWatcher library')
        print('future requests wait until the retry-after time passes')
    elif err.response.status_code == 404:
        print('Summoner with that ridiculous name not found.')
    else:
        raise

In [4]:
test = lol_watcher.champion_mastery.by_summoner(my_region, 'iqHggxw_J_90RXfoSzAqi9ITv-LmDdIFSFS-4z_qinnQnOY')
test_df = pd.DataFrame(test)

In [51]:
def get_summonerData(region, username):

        summoner = lol_watcher.summoner.by_name(my_region, username)
        summoner_df = pd.DataFrame([summoner])
        return summoner_df

summoner_df = get_summonerData(my_region, 'Doublelift')
summoner_df

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel
0,iqHggxw_J_90RXfoSzAqi9ITv-LmDdIFSFS-4z_qinnQnOY,ii2YY7ub39QQCOdaAMjv--q9Ql2IkzJ1K-rwcdPuZj9RBg,KYNPFp49bqq2Q30Uzu6OLohdycwn0Nm5Ai1hkiPCeY9DWO...,Doublelift,4568,1620947979000,251


In [7]:
def get_matchInfo(my_region, match_id):
    match_info = []
    stats = []
    stats_df = pd.DataFrame()
    
    for i in range(len(match_id)):
        data = lol_watcher.match.by_id(region = my_region, match_id = match_id[i])
        data_df = json_normalize(data)
    return data_df
doublelift_match_info = get_matchInfo(my_region, doublelift_match_ids)

In [18]:
doublelift_match_info

,gameId,platformId,gameCreation,gameDuration,queueId,mapId,seasonId,gameVersion,gameMode,gameType,teams,participants,participantIdentities
0,3911712447,NA1,1621379495211,1742,420,11,13,11.10.374.9538,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Win', 'firstBlood': F...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."


In [6]:
def get_matchId(my_region, summoner_df, num_matches):
    account_id = summoner_df['accountId'][0]
    match = lol_watcher.match.matchlist_by_account(region=my_region, encrypted_account_id=account_id, end_index=num_matches, season=13)
    match_df = pd.DataFrame(match['matches'])
    match_id = match_df['gameId']
    return match_id

doublelift_match_ids = get_matchId(my_region, summoner_df, 2)
doublelift_match_ids

0    3911698729
1    3911712447
Name: gameId, dtype: int64

In [6]:
def get_stats(my_region, match_id):
    match_info = []
    stats = []
    stats_df = pd.DataFrame()
    
    for i in range(len(match_id)):
        data = lol_watcher.match.by_id(region = my_region, match_id = match_id[i])
        data_df = json_normalize(data)
        match_info = data_df['participants'].values.tolist() 
        match_df = pd.concat([pd.DataFrame(t) for t in match_info], axis=1)
        stats = match_df['stats'].values.tolist()
        temp_df = pd.concat([pd.DataFrame(s,index=[0]) for s in stats])
        stats_df = stats_df.append(temp_df)
    stats_df.reset_index(drop=True,inplace=True)    
    return stats_df

test = get_stats('na1',doublelift_match_ids)



In [7]:
test

,participantId,win,item0,item1,item2,item3,item4,item5,item6,kills,...,perk4Var3,perk5,perk5Var1,perk5Var2,perk5Var3,perkPrimaryStyle,perkSubStyle,statPerk0,statPerk1,statPerk2
0,1,True,1056,6655,3157,1052,0,3020,3340,4,...,0,8347,0,0,0,8200,8300,5005,5008,5002
1,2,True,3006,0,0,3046,6676,6671,3363,10,...,0,8304,9,0,0,8100,8300,5005,5008,5002
2,3,True,2031,6631,3857,2055,3047,3044,3364,5,...,0,8352,168,994,0,8100,8300,5005,5008,5002
3,4,True,3158,3068,1082,3001,1028,3076,3364,4,...,0,9111,1427,300,0,8400,8000,5007,5002,5001
4,5,True,3075,3047,1028,6662,0,1028,3340,2,...,5,8345,3,0,0,8400,8300,5008,5002,5002
5,6,False,1055,3006,1036,3086,6672,1036,3363,0,...,0,8014,180,0,0,8100,8000,5005,5008,5002
6,7,False,3860,3190,2055,3047,1028,0,3364,2,...,0,8347,0,0,0,8400,8300,5007,5002,5003
7,8,False,1055,6673,3086,1018,0,3006,3363,4,...,0,8135,2148,5,0,8000,8100,5005,5008,5002
8,9,False,6630,0,1037,3071,3158,0,3364,7,...,0,8135,3167,3,0,8000,8100,5008,5008,5002
9,10,False,2033,6656,4629,2055,3158,3070,3340,3,...,0,8347,0,0,0,8200,8300,5008,5008,5002


In [10]:
def get_summonerName(my_region, match_id):
    match_info = []
    name_df = pd.DataFrame()
    
    for i in range(len(match_id)):
        data = lol_watcher.match.by_id(region = my_region, match_id = match_id[i])
        data_df = json_normalize(data['teams'])
        name_df = name_df.append(data_df)
        
#     name_df = name_df.drop(columns=['participantId','player.currentPlatformId','player.currentAccountId','player.matchHistoryUri','player.profileIcon','player.platformId'])    
    name_df.reset_index(drop=True,inplace=True)
    return name_df

test1 = get_summonerName('na1',doublelift_match_ids)

In [33]:
def get_summonerPuuid(region, summoner_df):

        puuid = lol_watcher.summoner.by_puuid(region, summoner_df['puuid'][0] )
        puuid_df = pd.DataFrame([puuid])
        return puuid_df
doublelift_puuid = get_summonerPuuid(my_region, summoner_df)
doublelift_puuid

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel
0,iqHggxw_J_90RXfoSzAqi9ITv-LmDdIFSFS-4z_qinnQnOY,ii2YY7ub39QQCOdaAMjv--q9Ql2IkzJ1K-rwcdPuZj9RBg,KYNPFp49bqq2Q30Uzu6OLohdycwn0Nm5Ai1hkiPCeY9DWO...,Doublelift,4568,1620947979000,251


In [46]:
please=lol_watcher.league.by_summoner('na1',summoner_df['id'][0])
please

[{'leagueId': '2bd81059-a0ce-33d4-9fb0-86977c851f64',
  'queueType': 'RANKED_SOLO_5x5',
  'tier': 'CHALLENGER',
  'rank': 'I',
  'summonerId': 'iqHggxw_J_90RXfoSzAqi9ITv-LmDdIFSFS-4z_qinnQnOY',
  'summonerName': 'Doublelift',
  'leaguePoints': 981,
  'wins': 207,
  'losses': 178,
  'veteran': False,
  'inactive': False,
  'freshBlood': False,
  'hotStreak': False}]

In [52]:
temp = pd.DataFrame(lol_watcher.league.by_summoner('na1',summoner_df['id'][0]))
rank = temp[['tier','rank']].copy()
summoner_df = summoner_df.join(rank, how='left')
summoner_df

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel,tier,rank
0,iqHggxw_J_90RXfoSzAqi9ITv-LmDdIFSFS-4z_qinnQnOY,ii2YY7ub39QQCOdaAMjv--q9Ql2IkzJ1K-rwcdPuZj9RBg,KYNPFp49bqq2Q30Uzu6OLohdycwn0Nm5Ai1hkiPCeY9DWO...,Doublelift,4568,1620947979000,251,CHALLENGER,I


In [20]:
df = test1.join(test,how='left')
df

,player.accountId,player.summonerName,player.summonerId,participantId,win,item0,item1,item2,item3,item4,item5,item6,kills,deaths,assists,largestKillingSpree,largestMultiKill,killingSprees,longestTimeSpentLiving,doubleKills,tripleKills,quadraKills,pentaKills,unrealKills,totalDamageDealt,magicDamageDealt,physicalDamageDealt,trueDamageDealt,largestCriticalStrike,totalDamageDealtToChampions,magicDamageDealtToChampions,physicalDamageDealtToChampions,trueDamageDealtToChampions,totalHeal,totalUnitsHealed,damageSelfMitigated,damageDealtToObjectives,damageDealtToTurrets,visionScore,timeCCingOthers,totalDamageTaken,magicalDamageTaken,physicalDamageTaken,trueDamageTaken,goldEarned,goldSpent,turretKills,inhibitorKills,totalMinionsKilled,neutralMinionsKilled,neutralMinionsKilledTeamJungle,neutralMinionsKilledEnemyJungle,totalTimeCrowdControlDealt,champLevel,visionWardsBoughtInGame,sightWardsBoughtInGame,wardsPlaced,wardsKilled,firstBloodKill,firstBloodAssist,firstTowerKill,firstTowerAssist,firstInhibitorKill,firstInhibitorAssist,combatPlayerScore,objectivePlayerScore,totalPlayerScore,totalScoreRank,playerScore0,playerScore1,playerScore2,playerScore3,playerScore4,playerScore5,playerScore6,playerScore7,playerScore8,playerScore9,perk0,perk0Var1,perk0Var2,perk0Var3,perk1,perk1Var1,perk1Var2,perk1Var3,perk2,perk2Var1,perk2Var2,perk2Var3,perk3,perk3Var1,perk3Var2,perk3Var3,perk4,perk4Var1,perk4Var2,perk4Var3,perk5,perk5Var1,perk5Var2,perk5Var3,perkPrimaryStyle,perkSubStyle,statPerk0,statPerk1,statPerk2
0,da34MJFERTZMbiueCAMp-FOTRImWTMQ98anLiDIK4IrE9t...,SugiKim,3v5XGdKW0SXu-svoTvDEu6DEnT3-mvEY7T-nxyuwPXe0AZ...,1,True,1056,6655,3157,1052,0,3020,3340,4,2,13,4,1,1,452,0,0,0,0,0,124848,112763,11126,958,0,20926,20236,690,0,0,0,11151,4617,2655,23,33,10922,3861,6454,606,10534,8410,0,0,189,18,12,2,361,15,3,0,11,3,False,False,False,False,False,True,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8214,1616,0,0,8226,250,1098,0,8210,19,0,0,8237,627,0,0,8345,3,0,0,8347,0,0,0,8200,8300,5005,5008,5002
1,7KaotGB4kTYIixKupIutZsoa3TZjPNPQJjmQGydrmaOGwI...,TL Tactical,Yi_69fJjpeiyA-zoJBzTRWep_skUiqL0JO4tIKJUR4h_vd...,2,True,3006,0,0,3046,6676,6671,3363,10,3,8,6,2,3,355,3,0,0,0,0,114067,8876,105032,157,440,15077,3026,11892,157,4682,4,6724,11745,5357,29,0,12037,2819,8498,719,12898,10775,2,1,203,12,0,4,0,13,5,0,13,9,True,False,False,False,True,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9923,52,78,0,8139,779,0,0,8138,18,0,0,8135,2280,5,0,8345,3,0,0,8304,9,0,0,8100,8300,5005,5008,5002
2,jNf4X2GSR97XMOiOCZ9tCS_W9HqJP8hPmliDnArVrE-5yJ...,From Iron,zZKToqfCmNd7UTvsIOQmTMLc5tS2CRWZ0lPBXiefBBssBXrm,3,True,2031,6631,3857,2055,3047,3044,3364,5,6,9,2,2,1,340,1,0,0,0,0,37596,0,28892,8703,0,10765,0,9708,1056,1164,1,12319,4813,4546,69,27,15113,3580,11153,378,8020,6800,1,0,32,0,0,0,99,11,8,0,39,7,False,False,False,False,False,True,0,0,0,0,0,0,0,1,0,0,0,0,0,0,9923,41,91,0,8126,398,0,0,8136,20,30,0,8105,13,5,0,8306,10,0,0,8352,168,994,0,8100,8300,5005,5008,5002
3,iVJddGNhzsW5WPDpM4GSDBF6vu3TmzAdniicGHKgn2zZZW...,pspspspsmeowmeow,YcwF5hr5accBvq5VVWUajUmBIvokhmHByuDvyfRbInzZesut,4,True,3158,3068,1082,3001,1028,3076,3364,4,4,11,2,1,1,710,0,0,0,0,0,104825,83086,15683,6055,14,12402,11078,267,1056,28316,1,23405,5180,699,17,41,28161,6823,18309,3027,9375,9125,0,0,18,109,84,7,488,13,3,0,4,2,False,False,False,False,False,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8439,750,0,0,8446,566,0,0,8429,50,16,15,8453,2576,0,0,9105,15,10,0,9111,1427,300,0,8400,8000,5007,5002,5001
4,L9Jz_EKSlCcRtdknpoBw3Xf4InUpav92ZdNdZpUCo4vGhg,Spawwwwn,DOgqFYL7-8EQnDKgpVSPJrCNvxJSYjO7YOmweqf0uf244wM,5,True,3075,3047,1028,6662,0,1028,3340,2,1,6,0,1,0,1109,0,0,0,0,0,92153,28092,64060,0,16,8180,3595,4585,0,1420,1,23564,8544,6888,15,19,15152,5513,9060,577,9148,7750,4,0,147,0,0,0,630,13,2,0,10,1,False,False,True,False,False,True,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8437,595,431,0,8446,3310,0,0,8473,527,0,0,8242,37,0,0,8304,11,1,5,8345,3,0,0,8400,8300,5008,5002,5002
5,ii2YY7ub39QQCOdaAMjv--q9Ql2IkzJ1K-rwcdPuZj9RBg,Doublelift,iqHggxw_J_90RXfoSzAqi9ITv-LmDdIFSFS-4z_qinnQnO

In [15]:
pd.set_option('display.max_columns', 150)

In [34]:
final_df = df[['player.accountId','player.summonerName','player.summonerId','participantId','win','kills','deaths','assists','largestKillingSpree','largestMultiKill','totalDamageDealtToChampions','magicDamageDealtToChampions','physicalDamageDealtToChampions','visionScore','totalMinionsKilled']].copy()


In [38]:
final_df = final_df.groupby('player.summonerName')[['kills','deaths','assists']].mean()
final_df

,kills,deaths,assists
player.summonerName,,,
100 Closer,4.0,6.0,20.0
ADCADC123,14.0,6.0,17.0
Asyc,7.0,10.0,6.0
Chase,7.0,3.0,2.0
Dannny,2.0,7.0,2.0
Doublelift,1.5,6.5,16.0
From Iron,5.0,6.0,9.0
GAKHWA,5.0,4.0,13.0
GeneralSn1per,7.0,10.0,9.0


KeyError: ('kills', 'deaths', 'assists')